# **Preparations for Meeting Simulation**

In [1]:
import numpy as np
import pandas as pd

import json

from multiprocessing.pool import Pool, ThreadPool
from multiprocessing import Lock

from tqdm.notebook import trange

import copy

import sumolib

In [2]:
SEEDS = [42, 1234, 1867, 613, 1001]
TIME_LIMIT = 300

In [3]:
RESULTS_ROOT = "../../02_data/01_simulation_results/"
VEH_LIST_PATH = "../../02_data/veh_list.json"
MEETING_PATH = "../../02_data/03_meeting_data/"
MEETING_VEHICLES = "../../02_data/meeting_vehicles"
EDGE_MAP_PATH = "../../02_data/edge_maps.json"
NEIGHBORING_EDGES_FILE = "../../02_data/neighboring_edges.json"

In [4]:
#reading *test* vehicles:

with open(VEH_LIST_PATH) as f:
    veh_list  = json.load(f)

test_vehicles = veh_list["test_vehs"]

In [5]:
def combine_commuters(veh_id):
    if veh_id.startswith("carIn"):
        return veh_id.split(":")[0]
    return veh_id

## Creating the combination dataset

In [ ]:
#reading parking simulation data and combine commuter vehicle ids:

#data reading:
p_data = pd.DataFrame()

for s in SEEDS:
    filename = f'{RESULTS_ROOT}/poccup_by_vehs_{s}.csv'
    pf = pd.read_csv(filename)
    pf["seed"] = [s]*len(pf)
    p_data = pd.concat([p_data, pf])

#handling the commuters:
p_data["veh_id"] = p_data["veh_id"].apply(combine_commuters)

In [ ]:
#creating key for the measurements:

def calculate_hash(datarow):
    key = r.parking_id + str(r.time) + str(r.occupancy) + str(r.seed)
    return hash(key)


hashes = []
for i,r in p_data.iterrows():
    hashes.append(calculate_hash(r))
p_data["hash"] = hashes

In [ ]:
p_data.to_csv(f"{MEETING_PATH}/combined_dataset.csv", index=False)

## Collecting neighboring edges

In [22]:
#input:
NETWORK_FILE = "../../01_simulation/02_scenario/rand_grid.net.xml"
#output:
NEIGHBORING_EDGE_FILE = "../../02_data/neighboring_edges.json"

In [17]:
def get_opposed(edge_name):
    opposed = ""
    if edge_name.startswith("-"):
        opposed = edge_name.split("-")[-1]
    else:
        opposed = f"-{edge_name}"
    return opposed

In [26]:
net = sumolib.net.readNet(NETWORK_FILE)
edges = net.getEdges()
neighbors = {}

#collecting neighbors:
for edge in edges:
    edge_id = edge.getID()
    neighbors[edge_id] = []
    for inc in edge.getIncoming():
        neighbors[edge_id].append(inc.getID())
        neighbors[edge_id].append(get_opposed(inc.getID()))
    for outg in edge.getOutgoing():
        neighbors[edge_id].append(outg.getID())
        neighbors[edge_id].append(get_opposed(outg.getID()))
        
#a couple of edges are connected to each other,
#therefore, we have to remove them from it neighbors:
for edge in neighbors:
    if edge in neighbors[edge]:
        neighbors[edge].remove(edge)

In [28]:
with open(NEIGHBORING_EDGE_FILE, "w") as f:
    json.dump(neighbors, f)

## Collecting meeting vehicles

In [6]:
#reading *test* vehicles:
with open(VEH_LIST_PATH) as f:
    veh_list  = json.load(f)

test_vehicles = veh_list["test_vehs"]

with open(EDGE_MAP_PATH) as f:
    edge_maps = json.load(f)

edge_to_idx = edge_maps["edge_to_idx"]

with open(NEIGHBORING_EDGES_FILE) as f:
    neighbors = json.load(f)

In [7]:
def combine_commuters(veh_id):
    if veh_id.startswith("carIn"):
        return veh_id.split(":")[0]
    return veh_id

In [8]:
#reading moving simulation data:

m_data = pd.DataFrame()
for s in SEEDS:
    filename = f"{RESULTS_ROOT}/vehicle_positions_{s}.csv"
    mf = pd.read_csv(filename)
    mf["seed"] = [s]*len(mf)
    m_data = pd.concat([m_data, mf])

m_data["veh_id"] = m_data["veh_id"].apply(combine_commuters)
m_data = m_data[m_data["veh_id"].isin(test_vehicles)]
m_data = m_data[~m_data["edge"].str.startswith(":")]

### Meeting definition functions

In [9]:
def collect_meeting_vehicles_narrow(m_data, meeting_time, meeting_times, meeting_time_gap=TIME_LIMIT):
    #collecting recently met vehicles:
    
    meetings = m_data[m_data["time"] == meeting_time]

    mets = []

    #same edges:
    for edge in edge_to_idx:
        vehs = meetings[meetings["edge"] == edge]["veh_id"].unique()
        for i in range(len(vehs)):
            for j in range(i+1, len(vehs)):
                sender = vehs[i]
                receiver = vehs[j]
                #if not met yet or met long time ago:
                if ((not((sender, receiver) in meeting_times)) or
                    meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                    mets.append((sender, receiver))
                    mets.append((receiver, sender)) #they meet vice-versa

    #opposed edges:
    for edge in edge_to_idx:
        #only "reversed" edges are processed, to avoid duplicated meetings:
        if edge.startswith("-"):
            veh_edge = meetings[meetings["edge"] == edge]["veh_id"].unique()
            contra_edge = edge.split("-")[1]
            veh_contra = meetings[meetings["edge"] == contra_edge]["veh_id"].unique()
            for sender in veh_edge:
                for receiver in veh_contra:
                    #if not met yet or met long time ago:
                    if ((not((sender, receiver) in meeting_times)) or
                        meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                        mets.append((sender, receiver))
                        mets.append((receiver, sender)) #they meet vice-versa


    return mets #vehicles at the same time, at the same place, not the 'ego' vehicle and not met recently

In [10]:
def collect_meeting_vehicles_broad(m_data, meeting_time, meeting_times, meeting_time_gap=TIME_LIMIT,
                                   neighboring_edges = neighbors):
    #collecting recently met vehicles:
    
    meetings = m_data[m_data["time"] == meeting_time]

    mets = []
        
    for edge in edge_to_idx:
        veh_edge = meetings[meetings["edge"] == edge]["veh_id"].unique()
        #same edge:
        for i in range(len(veh_edge)):
            for j in range(i+1, len(veh_edge)):
                sender = veh_edge[i]
                receiver = veh_edge[j]
                #if not met yet or met long time ago:
                if ((not((sender, receiver) in meeting_times)) or
                    meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                    mets.append((sender, receiver))
                    mets.append((receiver, sender)) #they meet vice-versa
        
        #neighboring edges:
        veh_contra = meetings[meetings["edge"].isin(neighboring_edges[edge])]["veh_id"].unique()
        for sender in veh_edge:
            for receiver in veh_contra:
                #if not met yet or met long time ago:
                if ((not((sender, receiver) in meeting_times)) or
                    meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                    mets.append((sender, receiver)) #vice-versa meetings are coming from the other neighboring


    return mets #vehicles at the same time, at the same place, not the 'ego' vehicle and not met recently

In [11]:
#running the collector scripts:
def collector_script(args):
    seed, collect_method, meas_name = args
    meeting_times = {}
    meeting_collection = {}

    m_data_filt = m_data[m_data["seed"] == seed]
    m_data_hour = m_data_filt[m_data_filt["time"] <= min(m_data_filt["time"])+6*60*60]
    
    for t in range(int(min(m_data["time"])), int(max(m_data["time"]))):
        if t%(6*60*60) == 0:
            m_data_hour = m_data_filt[(m_data_filt["time"] >= t) &
                            (m_data_filt["time"] < t+6*60*60)]
        #m_data_filt = m_data_filt[m_data_filt["time"] >= t]
        meetings = collect_method(m_data_hour, t, meeting_times)
        meeting_collection[t] = meetings
        for sender, receiver in meetings:
            meeting_times[(sender, receiver)] = t

    with open(f"{MEETING_VEHICLES}_{meas_name}_{seed}.json", "w") as f:
        json.dump(meeting_collection, f)

In [ ]:
with Pool(5) as ps:
    ps.map(collector_script, zip(SEEDS, [collect_meeting_vehicles_broad]*5, ["broad"]*5))